In [1]:
include("src/makematrix.jl")

Our example has 5 counties in a row, with links between them.
Resources are produced in counties at a county-specific cost, and then can be transported to other counties.
Each county has a resource requirement, and the problem is to satisfy that requirement for minimum cost.

In [2]:
# How much resource each county requires
requirements = [1, 2, 3, 4, 5];
# The cost per unit of producing it within each county
productioncosts = [1, 3, 5, 7, 9];
# The cost of transporting a unit of resource to the next county
transportcost = 2;

Now we set up the objective and constraints.  There are 5 constraints for the mass-balance of each county and the variables are production in each county (5 values) and transportation between the counties (4 values).

In [3]:
# Minimize the production + transportation costs
objective(production, transport) =
    sum(production .* productioncosts) + sum(transport * transportcost)

# Make a network constraint for county ii
function makeconstraint(ii)
    # The constraint function
    function constraint(production, transport)
        transport = [0; transport; 0] # force no transport on boundary
        # Require that production - exports + imports >= requirements
        requirements[ii] - (production[ii] - transport[ii + 1] + transport[ii])
    end
end

# Set up the constraints
constraints = map(makeconstraint, 1:5);

To create the five constraints, we set up a `makeconstraint` function.  All that does is specify which county the constraint is for, and then the `constraint` function defined inside of it describes the mass-balance requirement:
\begin{align*}
  R_i \le & P_i - \sum_j T_{ij} - T_{ji} \\
  R_i \le & P_i - I_{i+1} + I_i \\
\end{align*}
where $R_i$ is the requirement in county $i$, $P_i$ is its production, and $T_{ij}$ is the transport from county $i$ to county $j$.  In our simplified example, we instead write this in terms of $I_i$, the import of county $i$ from its upstream neighbor.

Now we create the linear programming matrices, by evaluating the derivative at any point (since its a linear system).  The matrices are for the form:
\begin{equation*}
\min f' x \text{ such that $A x \le b$}
\end{equation*}

In [4]:
# Define a single point solution
x0 = zeros(9);

# Create the matrices for the linear programming problem
f = objectivevector(objective, x0, args=[5, 4])

9-element Array{Float64,1}:
 1.0
 3.0
 5.0
 7.0
 9.0
 2.0
 2.0
 2.0
 2.0

The objective function places a county-specific penalty on the 5 productions and 4 transport variables.

In [5]:
A, b = constraintmatrix(constraints, x0, args=[5, 4]);

In [6]:
A

5x9 Array{Float64,2}:
 -1.0  -0.0  -0.0  -0.0  -0.0   1.0  -0.0  -0.0  -0.0
 -0.0  -1.0  -0.0  -0.0  -0.0  -1.0   1.0  -0.0  -0.0
 -0.0  -0.0  -1.0  -0.0  -0.0  -0.0  -1.0   1.0  -0.0
 -0.0  -0.0  -0.0  -1.0  -0.0  -0.0  -0.0  -1.0   1.0
 -0.0  -0.0  -0.0  -0.0  -1.0  -0.0  -0.0  -0.0  -1.0

The $A$ matrix is a little difficult to read in this form because of all the `-0.0` values, but it describes the five constraints each in a row, with a `-1.0` for each unit of production and import and an `1.0` for each unit of export.  Remeber that the requirement is satisfied when it is $< 0$.

In [7]:
b

5-element Array{Float64,1}:
 -1.0
 -2.0
 -3.0
 -4.0
 -5.0

The $b$ vector defines the constants for the requirements.

Now we can solve it!  We specify that the solution lies between all productions and transports being 0 and being $\infty$.

In [8]:
# Solve it!
sol = linprog(f, A, '<', b, zeros(9), ones(9) * Inf)
sol.sol

Optimize a model with 5 rows, 9 columns and 13 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+00, 9e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 5e+00]
Presolve removed 5 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  9.500000000e+01


9-element Array{Float64,1}:
  1.0
 14.0
  0.0
  0.0
  0.0
  0.0
 12.0
  9.0
  5.0

So the solution is to have all the resource produced in county 1 and 2 and then transported to the rest.  Note that if the transport cost is $\le 1$, it will all be produced in county 1, and it the transport cost is $> 2$ it will be produced in each county individually.